<a href="https://colab.research.google.com/github/harukam122/ghiblifiy/blob/main/ghiblify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocess

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# cd drive/MyDrive/DL\ Final\ Project

In [ ]:
cd /content/drive/.shortcut-targets-by-id/12unBA6eflIDir_xYDi0aG-jbdDiDCrqf/DL Final Project

/content/drive/.shortcut-targets-by-id/12unBA6eflIDir_xYDi0aG-jbdDiDCrqf/DL Final Project


In [ ]:
import os
import torch
import torchvision.transforms as transforms
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

import IPython.display as display
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
from PIL import Image
import time
import functools
import torchvision.transforms.functional as TF

device=torch.device("cuda" if (torch.cuda.is_available()) else 'cpu')

In [ ]:
# content_url = "/content/drive/.shortcut-targets-by-id/12unBA6eflIDir_xYDi0aG-jbdDiDCrqf/DL Final Project/images/content_three.jpeg"
# style_url = "/content/drive/.shortcut-targets-by-id/12unBA6eflIDir_xYDi0aG-jbdDiDCrqf/DL Final Project/images/style_three.jpeg"
style_url = "/content/drive/Shared-with-me/DL Final Project/images/style_two.jpg"
content_url = "/content/drive/Shared-with-me/DL Final Project/images/content_two.jpg"
# style_url = "/content/drive/Shared with me/DL Final Project/images/style_two.jpg"
# content_url = "/content/drive/Shared with me/DL Final Project/images/content_two.jpeg"

# content_url = "/content/drive/MyDrive/DL Final Project/images/labrador_content.jpg"
# style_url = "/content/drive/MyDrive/DL Final Project/images/kandinsky_style.jpg"

# Define a function for normalizing the images
def normalize(img):
    return transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(img)

# Define a function for denormalizing the images
def denormalize(img):
    mean = torch.tensor([0.485, 0.456, 0.406]).cuda()
    std = torch.tensor([0.229, 0.224, 0.225]).cuda()
    return img * std[:, None, None] + mean[:, None, None]

def load_img(path_to_img):
    max_dim = 512
    img = Image.open(path_to_img).convert('RGB')
    width, height = img.size
    if height > width:
        new_height = max_dim
        new_width = int(max_dim * width / height)
    else:
        new_width = max_dim
        new_height = int(max_dim * height / width)
    img = transforms.Resize((new_height, new_width))(img)
    img = transforms.ToTensor()(img)
    img = normalize(img)
    img = img.unsqueeze(0) # add batch dimension
    return img.to(device,torch.float)

In [ ]:
def imshow(image, title=None):
  image = image.cuda().clone()
  if len(image.shape) > 3:
    image = image.squeeze(0)
  image = denormalize(image)
  image = image.detach().cpu().numpy()
  image = image.squeeze().transpose(1, 2, 0)
  image = image.clip(0,1)
  if title:
    plt.imshow(image, title)
  else:
    plt.imshow(image)

In [ ]:
content_image = load_img(content_url)
style_image = load_img(style_url)

plt.subplot(1, 2, 1)
imshow(content_image)

plt.subplot(1, 2, 2)
imshow(style_image)

FileNotFoundError: ignored

# Defining Content/Style Representations

In [ ]:
import torchvision.models as models
from torchvision.models.feature_extraction import create_feature_extractor

content_layers = ['31']
style_layers = ['1', '6', '11', '20', '29']

# content_layers = {
#     '30': 'block5_conv2'
#   }

# style_layers = {
#     '0': 'block1_conv1',
#     '5': 'block2_conv1',
#     '10': 'block3_conv1',
#     '19': 'block4_conv1',
#     '28': 'block5_conv1',
#   }

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

# Gram Matrix

In [ ]:
def gram_matrix(features):
    # first dimension is batch dimension
    _, c, h, w = features.size()
    features = features.view(c, h*w)
    return features@features.T

# Extract style and content

In [ ]:
class StyleContentModel(torch.nn.Module):
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.vgg = models.vgg19(pretrained=True).features.cuda().eval()
    for param in self.vgg.parameters():
      param.requires_grad = False

    self.layers = style_layers + content_layers
    self.extractor = create_feature_extractor(self.vgg, return_nodes=self.layers)

  def forward(self, inputs):
    feat_dict = self.extractor(inputs)
    features = list(feat_dict.values())
    
    # Sort layer outputs to style and content layers
    style_outputs, content_outputs = (features[:num_style_layers],
                                      features[num_style_layers:])
  

    # apply gram matrix
    style_outputs = [gram_matrix(output) for output in style_outputs]

    # Assign content and style representations as dictionaries
    content_dict = {content_name: value
                    for content_name, value
                    in zip(content_layers, content_outputs)}
    style_dict = {style_name: value
                  for style_name, value
                  in zip(style_layers, style_outputs)}

    return {'content': content_dict, 'style': style_dict}

In [ ]:
extractor = StyleContentModel(style_layers, content_layers)
results = extractor(content_image)

print('Styles:')
for name, output in sorted(results['style'].items()):
    print("  ", name)
    print("    shape: ", output.cpu().detach().numpy().shape)
    print("    min: ", output.cpu().detach().numpy().min())
    print("    max: ", output.cpu().detach().numpy().max())
    print("    mean: ", output.cpu().detach().numpy().mean())
    print()

print("Contents:")
for name, output in sorted(results['content'].items()):
    print("  ", name)
    # if output.shape[1] > 1:
    #     output = output[:, -2:-1, :, :]
    # imshow(output[0], title=name)
    print("    shape: ", output.cpu().detach().numpy().shape)
    print("    min: ", output.cpu().detach().numpy().min())
    print("    max: ", output.cpu().detach().numpy().max())
    print("    mean: ", output.cpu().detach().numpy().mean())

# Run Gradient Descent

In [ ]:
style_targets = extractor(style_image)['style']
content_targets = extractor(content_image)['content']

image = content_image.clone().requires_grad_(True)

In [ ]:
def clip_0_1(image):
    return torch.clamp(image, min=0.0, max=1.0)

In [ ]:
opt = torch.optim.Adam([image], lr=0.02, betas=(0.99, 0.999), eps=1e-1)

# Weighted combination of losses
style_weight=1e-2
content_weight=1e4

In [ ]:
import torch.nn.functional as F

def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']

    # Mean squared error for every GRAM MATRIX of style layer outputs
    style_loss = sum([F.mse_loss(style_outputs[name], style_targets[name]) 
                      for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    # Mean squared error for every content layer outputs
    content_loss = sum([F.mse_loss(content_outputs[name], content_targets[name]) 
                        for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers

    # Sum losses
    loss = style_loss + content_loss
    return loss

In [ ]:
def train_step(image):
    outputs = extractor(image)
    loss = style_content_loss(outputs)
    opt.zero_grad()
    loss.backward()
    opt.step()

    # Clip the image pixel values to be between 0 and 1
    clip_0_1(image)
    return loss

In [ ]:
train_step(image)
train_step(image)
train_step(image)

In [ ]:
# display content and final, target image
imshow(image)

In [ ]:
import time
start = time.time()

epochs = 10
steps_per_epoch = 100

step = 0
for n in range(epochs):
    for m in range(steps_per_epoch):
        step += 1
        loss = train_step(image)
        print(".", end='', flush=True)
    imshow(image)
    plt.show()
    print("Train step: {}".format(step) + " Loss: {}".format(loss))

end = time.time()
print("Total time: {:.1f}".format(end-start))

In [ ]:
import time
def transfer_style(gen_image, content_image, style_image, epochs = 10, steps_per_epoch = 100):
  opt = torch.optim.Adam([image], lr=learning_rate, betas=(0.99, 0.999), eps=1e-1)

  start = time.time()

  style_targets = extractor(style_image)['style']
  content_targets = extractor(content_image)['content']

  step = 0
  for n in range(epochs):
      for m in range(steps_per_epoch):
          step += 1
          loss = train_step(gen_img)
          print(".", end='', flush=True)
      imshow(gen_img)
      plt.show()
      print("Train step: {}".format(step) + " Loss: {}".format(loss))

  end = time.time()
  print("Total time: {:.1f}".format(end-start))

In [ ]:
# HYPER PARAMETERS
learning_rate = 0.02
epochs = 10
steps_per_epoch = 100
style_weight=1e-2
content_weight=1e4

# INPUTS
# content_url = "/content/drive/MyDrive/DL Final Project/images/content_img.jpg"
# style_url = "/content/drive/MyDrive/DL Final Project/images/ghibli_style.jpg"
content_url = "/content/drive/.shortcut-targets-by-id/12unBA6eflIDir_xYDi0aG-jbdDiDCrqf/DL Final Project/images/content_three.jpeg"
style_url = "/content/drive/.shortcut-targets-by-id/12unBA6eflIDir_xYDi0aG-jbdDiDCrqf/DL Final Project/images/style_three.jpeg"

In [ ]:
content_image = load_img(content_url)
style_image = load_img(style_url)
gen_img = content_image.clone().requires_grad_(True)

# DOESN'T WORK FOR SM REASN
transfer_style(gen_img, content_image, style_image, epochs, steps_per_epoch)